In [29]:
from pathlib import Path
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography import x509
import re

In [30]:
def egcd(a, b):
    """Algorithme d'Euclide étendu.
    Retourne (g, x, y) tels que g = gcd(a, b) et ax + by = g.
    """
    if b == 0:
        return a, 1, 0
    g, x1, y1 = egcd(b, a % b)
    return g, y1, x1 - (a // b) * y1

In [31]:
def inv_mod(a, n):
    """Inverse modulaire : trouve x tel que a*x ≡ 1 mod n."""
    g, x, y = egcd(a, n)
    if g != 1:
        raise ValueError("Pas d'inverse modulaire")
    return x % n

In [42]:
def compute_u_i(N1, N2, N3):
    N = N1 * N2 * N3

    N_hat1 = N // N1
    N_hat2 = N // N2
    N_hat3 = N // N3

    u1 = inv_mod(N_hat1 % N1, N1)
    u2 = inv_mod(N_hat2 % N2, N2)
    u3 = inv_mod(N_hat3 % N3, N3)

    return (u1, u2, u3), (N_hat1, N_hat2, N_hat3)


In [43]:
def compute_m_cubed(c1, c2, c3, N1, N2, N3):
    (u1, u2, u3), (Nh1, Nh2, Nh3) = compute_u_i(N1, N2, N3)
    N = N1 * N2 * N3

    m3 = (c1 * u1 * Nh1 +
          c2 * u2 * Nh2 +
          c3 * u3 * Nh3) % N

    return m3, N


In [44]:
def integer_cube_root(n: int) -> int:
    """Renvoie la racine cubique entière de n si elle est exacte, sinon lève une erreur.
       Version 100% entière (pas de float)."""
    if n < 0:
        raise ValueError("n doit être >= 0")

    low, high = 0, 1

    while high**3 <= n:
        high <<= 1

    while low <= high:
        mid = (low + high) // 2
        cube = mid**3

        if cube == n:
            return mid
        elif cube < n:
            low = mid + 1
        else:
            high = mid - 1

    raise ValueError("La racine cubique n'est pas entière exacte.")

In [45]:
def recover_message_from_broadcast(c1, c2, c3, N1, N2, N3):
    m3, N = compute_m_cubed(c1, c2, c3, N1, N2, N3)
    m = integer_cube_root(m3)
    return m


In [46]:
import base64
from pathlib import Path

def read_cipher_as_int(filename):
    data_b64 = Path(filename).read_bytes().strip()
    data_bytes = base64.b64decode(data_b64)
    return int.from_bytes(data_bytes, "big")

c0 = read_cipher_as_int("c0")
c1 = read_cipher_as_int("c1")
c2 = read_cipher_as_int("c2")


In [47]:
def read_modulus_from_file(filename: str):
    """
    Lit un fichier de clé/certificat RSA (PEM, DER ou format texte simple)
    et renvoie (N, e).

    Stratégie :
      1) essayer PEM/DER standard (clé publique ou certificat)
      2) sinon, lire comme texte et chercher un gros entier (décimal ou hex)
    """
    data = Path(filename).read_bytes()

    if b"BEGIN CERTIFICATE" in data:
        try:
            cert = x509.load_pem_x509_certificate(data, backend=default_backend())
            pubkey = cert.public_key()
            numbers = pubkey.public_numbers()
            return numbers.n, numbers.e
        except Exception:
            pass

    if b"BEGIN PUBLIC KEY" in data or b"BEGIN RSA PUBLIC KEY" in data:
        try:
            pubkey = serialization.load_pem_public_key(data, backend=default_backend())
            numbers = pubkey.public_numbers()
            return numbers.n, numbers.e
        except Exception:
            pass

    try:
        pubkey = serialization.load_der_public_key(data, backend=default_backend())
        numbers = pubkey.public_numbers()
        return numbers.n, numbers.e
    except Exception:
        try:
            cert = x509.load_der_x509_certificate(data, backend=default_backend())
            pubkey = cert.public_key()
            numbers = pubkey.public_numbers()
            return numbers.n, numbers.e
        except Exception:
            pass

    text = data.decode(errors="ignore")

    m_dec = re.search(r'N\\s*=?\\s*([0-9]{10,})', text)
    if m_dec:
        return int(m_dec.group(1)), 3

    for line in text.splitlines():
        line = line.strip()
        if re.fullmatch(r"[0-9]{10,}", line):
            return int(line), 3

    hex_candidates = re.findall(r"[0-9A-Fa-f]{64,}", text)
    if hex_candidates:
        return int(hex_candidates[0], 16), 3

    raise ValueError(f"Format de clé non reconnu pour {filename}")

In [48]:
N0, e0 = read_modulus_from_file("clef0_pub.pem")
N1, e1 = read_modulus_from_file("clef1_pub.pem")
N2, e2 = read_modulus_from_file("clef2_pub.pem")

print(N0, e0)
print(N1, e1)
print(N2, e2)

24088132627827854260227095241287626275732065067806834186623354160975432335738726761195386764521182300680771315112595887201110496947915341306995300639424916947195887557394756823646362931943509329576304869763822239754059209491009642827304173660157530461585166686801066985725892378374699903837941502096545364422456937171368348052512305352550642196383717659523753887771737636362716095040744434309074719750525888209779815280198441415524034624188190016783981969471718614753958068750629559832406702652702027110910387697750594631583852269075218123815223243656342016700606595192879304680108711721159527850434352951850296708923 3
23483254155775775350914711489190204913200836124886389892537831743880612865434321302823443546235326623021942749852377298438662923115032903783705403434215933729346997746015514508309888420123476842789519251747448724493073678460705389773618508950296849197809182217608071220208495545565631056563132075962498678584365497520211315359228195682784983061715682353084850669467137345436861965

In [49]:
m = recover_message_from_broadcast(c0, c1, c2, N0, N1, N2)
print("m =", m)


m = 8388131359720046063872833100811325574570587786581033027443036972651370077226249633121066183702174905601194465162357203577717819386494559177357917520660217340214386313732889000995729043726276593413446227015142952546721646496051966772341834642570390834410121150397884781773266075473849193791775472457384434191196865262762067763768895485761779044905909446640835467821924369760638688355298154242940763320223330690939894916560502053278601601747985596894083811725949290750309782142188283705393989948406653725691406575786268881626886111264816551591864627974415487282240166658800843761054383756438874711807710508977464811552


In [50]:
m_bytes = m.to_bytes((m.bit_length() + 7) // 8, "big")
print(m_bytes)

try:
    print(m_bytes.decode("utf-8"))
except:
    print("Décodage impossible (pas UTF-8).")


b'Bravo! Pour valider le challenge, la clef est NoSpamWithRSA!\nWell done, the key for th challenge is NoSpamWithRSA!\n                                                                                                                                             '
Bravo! Pour valider le challenge, la clef est NoSpamWithRSA!
Well done, the key for th challenge is NoSpamWithRSA!
                                                                                                                                             
